In [ ]:
# Cells 4 onwards produce different outputs. Only run the cell(s) for the output(s) you want.

# this version only takes every nth post

In [35]:
import csv
import pandas as pd
import csv
import re
import string
import os
import nltk
import numpy as np
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OMars\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
pip install -r cluster-requirements.txt  #might be some missing, let me know if so

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#might'


In [10]:
# Change csv_name, which we're expecting to be stored in the 'data' subdir. 
# Needs fields 'twitter.tweet/text' and 'twitter.tweet/created'
csv_name = 'post-issues-w-stakeholders-for-cluster.csv'
text_field = 'twitter.tweet/text'

data_dir = os.path.join(os.getcwd(), 'data')
output_dir = os.path.join(data_dir, 'outputs')
csv_path = os.path.join(data_dir, csv_name)

with open(csv_path, "r", encoding='utf-8') as csv_f:
    reader = csv.DictReader(csv_f)
    doclengths = [len(row[text_field]) for row in reader]

row_count = len(doclengths)
avg_doc_size = int(sum(doclengths) / row_count)

print("{} rows in {}".format(row_count, csv_name))
print("Average document length is {}".format(avg_doc_size))

# Define a default for n - remove to prompt every time
n = 50
if not n:  
    n = input('Sample every nth cell, where n = ')
    n = int(n)
print("sampling 1 in every {} row(s)".format(n))

# create pandas dataframe from only every nth row from csv - 
# taken from https://stackoverflow.com/questions/53812094/select-every-nth-row-as-a-pandas-dataframe-without-reading-the-entire-file

skip = np.arange(row_count)
skip = np.delete(skip, np.arange(0, row_count, n))
df = pd.read_csv(csv_path, skiprows = skip)
df.index.name = 'id'
print('df created')

TypeError: string indices must be integers

In [5]:
df

,classify/post-classifier-v3-29-03-18-issue,classify/post-classifier-v3-29-03-18-not-issue,stakeholder/APC,stakeholder/APC-matches,stakeholder/Amazon,stakeholder/Amazon-matches,stakeholder/Collect-Plus,stakeholder/Collect-Plus-matches,stakeholder/DHL,stakeholder/DHL-matches,...,stakeholder/UK-Mail-matches,stakeholder/UPS,stakeholder/UPS-matches,stakeholder/Yodel,stakeholder/Yodel-matches,stakeholder/iPostParcels,stakeholder/iPostParcels-matches,twitter.tweet/created,twitter.tweet/id,twitter.tweet/text
id,,,,,,,,,,,,,,,,,,,,,
0,issue,NaN,False,NaN,False,NaN,False,NaN,False,NaN,...,NaN,False,NaN,False,NaN,False,NaN,2021-01-01T08:00:53.000Z,1344916490655592448,Testing REST APIs using Postman free download ...
1,issue,NaN,False,NaN,False,NaN,False,NaN,False,NaN,...,NaN,False,NaN,False,NaN,False,NaN,2021-01-01T10:16:44.000Z,1344950680411365379,For the last two weeks @RoyalMailHelp have sai...
2,issue,NaN,False,NaN,False,NaN,False,NaN,False,NaN,...,NaN,False,NaN,False,NaN,False,NaN,2021-01-01T11:03:44.000Z,1344962505643335681,@RoyalMail Oh the sunlit uplands
3,issue,NaN,False,NaN,False,NaN,False,NaN,False,NaN,...,NaN,False,NaN,False,NaN,False,NaN,2021-01-01T11:48:23.000Z,1344973741533573121,@Linda_Marric @parcelforce At the risk of bomb...
4,issue,NaN,False,NaN,False,NaN,False,NaN,False,NaN,...,NaN,False,NaN,False,NaN,False,NaN,2021-01-01T12:24:19.000Z,1344982784905981952,"@PostOffice Happy New Year, I paid £7.50 for n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8023,issue,NaN,False,NaN,False,NaN,False,NaN,False,NaN,...,NaN,False,NaN,False,NaN,False,NaN,2021-10-28T11:59:43.000Z,1453692956624527365,@hermesparcels for the last 3 days my parcel...
8024,issue,NaN,False,NaN,False,NaN,False,NaN,False,NaN,...,NaN,False,NaN,False,NaN,False,NaN,2021-10-28T12:19:43.000Z,1453697987524300805,@Hermesparcels how on earth do i speak to a hu...
8025,issue,NaN,False,NaN,False,NaN,False,NaN,False,NaN,...,NaN,False,NaN,False,NaN,False,NaN,2021-10-28T12:41:38.000Z,1453703503008915459,@connollyjon @Hermesparcels Had the misfortune...


In [48]:
#OPTION 1 Produces one file with one header

output = input('filename output? ')
output = output + '.txt'

a = list(df['twitter.tweet/text'])
b = '\n'.join(str(t) for t in a)

b = re.sub(r'[^a-zA-Z \n]+','', b)
#b = os.linesep.join([s for s in b.splitlines() if s])
#print(b)


with open(output, 'w', encoding = 'utf-8') as f:
    
    f.write('**** *all_text\n\n')
    f.write(b.lower())



filename output? 10000


In [12]:
#option 2 : produce one file with separated years

df['date'] = df['twitter.tweet/created'].str[:4]

with open('post-cluster-test.txt', 'w', encoding = 'utf-8') as f:

    for year in range (2017, 2022):
        year_str = str(year) 
        f.write('**** *year_'+year_str+'\n\n')
        df_temp = df[df['date'] == year]
        text = list(df['twitter.tweet/text'])
        text = '\n'.join(str(t) for t in text)
        text = re.sub(r'[^a-zA-Z \n]+','', text)
        f.write(text.lower())
        f.write('\n\n')

In [49]:
#option 3 : produce separate file for each year

df_1 = pd.read_csv(r'C:\Users\OMars\Dropbox\Oliver Sharing Stuff\2021 Work\CASM\Citizens-Advice\post-clean-for-clustering.csv')
df['date'] = df['twitter.tweet/created'].str[:4]


for year in range (2017, 2022):
    year_str = str(year)
    doc_title = 'post-cluster-'+year_str+'-100k-only-1.txt'
    with open(doc_title, 'w', encoding = 'utf-8') as f:
        f.write('**** *year_'+year_str+'\n\n')
        df_temp = df[df['date'] == year]
        text = list(df['twitter.tweet/text'])
        text = '\n'.join(str(t) for t in text)
        text = re.sub(r'[^a-zA-Z \n]+','', text)
        f.write(text.lower())

In [56]:
#option 4: one file with separated years and stopwords removed

#define stopword remover
# NLTK Stop words

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

df['date'] = df'twitter.tweet/created'].str[:4]

with open('post-cluster-all-years-nostops-1.txt', 'w', encoding = 'utf-8') as f:

    for year in range (2017, 2022):
        year_str = str(year) 
        f.write('**** *year_'+year_str+'\n\n')
        df_temp = df[df['date'] == year]
        text = list(df['twitter.tweet/text'])
        text = [word for word in text if word not in stopwords.words('english')]
        text = '\n'.join(str(t) for t in text)
        text = re.sub(r'[^a-zA-Z \n]+','', text)
        f.write(text.lower())
        f.write('\n\n')

In [9]:
# Option 5: split by companies.
# Creates two types of files:
# An 'everything file', containing all mentions of any company...
everything_filename = "all_text_by_companies.txt"
# ...and a bunch of company specific files, containing mentions of each.
company_filename_template = "company_mentions_{}.txt"

companies = ('APC', 'Amazon', 'Collect-Plus', 'DHL', 'DPD', 'DX', 'Doddle', 
             'FedEx', 'Hermes', 'HubBox', 'InPost', 'Interparcel', 'P4D', 
             'Parcel-Broker', 'Parcel-Monkey', 'Parcel2Go', 'ParcelCompare', 
             'ParcelHero', 'Parcelforce', 'Parcelly', 'Pass-My-Parcel', 
             'Post-Office', 'Royal-Mail', 'TNT', 'Tuffnells', 'UK-Mail', 
             'UPS', 'Yodel', 'iPostParcels')


# Create a new dataframe which only has matches plus text
def write_company_data_to_file(f):
    f.write('**** *company_'+company.lower()+'\n\n')
    text = list(df_3['twitter.tweet/text'])
    text = '\n'.join(str(t) for t in text)
    text = re.sub(r'[^a-zA-Z \n ]+','', text)
    f.write(text.lower())
    f.write('\n\n')

for i, company in enumerate(companies):

    # Look for a boolean field written by M52
    company_tag = company+'-matches'
    
    # Pick out rows from df (created above) which contain our tag
    df_matches = df.loc[:,df.columns.str.endswith(company_tag)]

    df_2 = pd.concat([df["twitter.tweet/text"], df_matches], axis=1)
    df_2.rename(columns = {list(df_2)[1]: 'match-column'}, inplace = True)
    
    df_3 = df_2[~df_2['match-column'].isnull()]

    # Append to everything file (or overwrite if this is the first company)
    if i == 0:
        write_mode = 'w'
    else: 
        write_mode = 'a'

    with open(os.path.join(output_dir, everything_filename), write_mode) as ef:
        write_company_data_to_file(ef)
    
    # Write to company specific file
    company_filename = company_filename_template.format(company)
    with open(os.path.join(output_dir, company_filename), 'w') as cf:
        write_company_data_to_file(cf)